## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-21 03:44:32.120892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 03:44:32.862285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "axion1"

## Loading data

See `cnn/data.py`.

In [6]:
with open(f"{data_dir}/processed/cnn/{task_name}_X_train.pkl", "rb") as fin:
    X_train = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_Y_train.pkl", "rb") as fin:
    Y_train = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_X_test.pkl", "rb") as fin:
    X_test = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_Y_test.pkl", "rb") as fin:
    Y_test = pickle.load(fin)

In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (210000, 4, 16, 1) (210000, 4, 128, 1)
  labels: (210000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-21 03:44:52.207084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-01-21 03:44:52.213240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9804 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2024-01-21 03:44:52.215566: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9804 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3d:00.0, compute capability: 7.5
2024-01-21 03:44:52.217266: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9804 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
visible3 = Input(shape=(16, 16, 1))
conv31 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible3)
pool31 = MaxPooling2D(pool_size=(2, 2))(conv31)
conv32 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool31)
pool32 = MaxPooling2D(pool_size=(2, 2))(conv32)
flat3 = Flatten()(pool32)
print ('cov31.shape', conv31.shape)
print ('poo31.shape', pool31.shape)
print ('cov32.shape', conv32.shape)
print ('poo32.shape', pool32.shape)

cov31.shape (None, 16, 16, 32)
poo31.shape (None, 8, 8, 32)
cov32.shape (None, 8, 8, 16)
poo32.shape (None, 4, 4, 16)


In [11]:
visible4 = Input(shape=(16, 8, 1))
conv41 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible4)
pool41 = MaxPooling2D(pool_size=(2, 2))(conv41)
conv42 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool41)
pool42 = MaxPooling2D(pool_size=(2, 2))(conv42)
flat4 = Flatten()(pool32)
print ('cov41.shape', conv41.shape)
print ('poo41.shape', pool41.shape)
print ('cov42.shape', conv42.shape)
print ('poo42.shape', pool42.shape)

cov41.shape (None, 16, 8, 32)
poo41.shape (None, 8, 4, 32)
cov42.shape (None, 8, 4, 16)
poo42.shape (None, 4, 2, 16)


In [12]:
merge = concatenate([flat1, flat2, flat3, flat4])

In [13]:
# interpretation model
hidden1 = Dense(20, activation='relu')(merge)
hidden2 = Dense(20, activation='relu')(hidden1)
output = Dense(3)(hidden2)
cnn = Model(inputs=[visible1, visible2, visible3, visible4], outputs=output)

In [14]:
# summarize layers
# print(cnn.summary())

In [15]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 16, 16, 1) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 16, 8, 1) dtype=float32 (created by layer 'input_4')>]

## Train the model

In [16]:
def train_iteration(lr, epochs):
    print(f"=== Training with lr={lr} for {epochs} epochs ===")
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=128,
        validation_data=(X_test, Y_test),
    )

In [17]:
train_iteration(lr=1e-2, epochs=5)

=== Training with lr=0.01 for 5 epochs ===
Epoch 1/5


2024-01-21 03:44:57.656959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2024-01-21 03:44:57.882317: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-21 03:44:58.263834: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa1e5ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-21 03:44:58.263933: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-21 03:44:58.263957: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-21 03:44:58.263978: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-21 03:44:58.264002: I tensorflow/compiler/xla/service/service.cc:176]  

1641/1641 [==============================] - 18s 8ms/step - loss: 8.6761 - accuracy: 0.9392 - val_loss: 0.0747 - val_accuracy: 0.9779
Epoch 2/5
1641/1641 [==============================] - 11s 7ms/step - loss: 0.0834 - accuracy: 0.9753 - val_loss: 0.0716 - val_accuracy: 0.9789
Epoch 3/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0785 - accuracy: 0.9766 - val_loss: 0.0984 - val_accuracy: 0.9678
Epoch 4/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0961 - accuracy: 0.9709 - val_loss: 0.0879 - val_accuracy: 0.9736
Epoch 5/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.1292 - accuracy: 0.9595 - val_loss: 0.1361 - val_accuracy: 0.9550


In [18]:
train_iteration(lr=2e-3, epochs=5)

=== Training with lr=0.002 for 5 epochs ===
Epoch 1/5
1641/1641 [==============================] - 12s 6ms/step - loss: 0.0952 - accuracy: 0.9709 - val_loss: 0.0899 - val_accuracy: 0.9726
Epoch 2/5
1641/1641 [==============================] - 7s 5ms/step - loss: 0.0860 - accuracy: 0.9737 - val_loss: 0.0819 - val_accuracy: 0.9755
Epoch 3/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0804 - accuracy: 0.9755 - val_loss: 0.1001 - val_accuracy: 0.9664
Epoch 4/5
1641/1641 [==============================] - 9s 5ms/step - loss: 0.0808 - accuracy: 0.9754 - val_loss: 0.0800 - val_accuracy: 0.9760
Epoch 5/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0758 - accuracy: 0.9770 - val_loss: 0.0781 - val_accuracy: 0.9769


In [19]:
train_iteration(lr=1e-3, epochs=5)

=== Training with lr=0.001 for 5 epochs ===
Epoch 1/5
1641/1641 [==============================] - 12s 6ms/step - loss: 0.0681 - accuracy: 0.9798 - val_loss: 0.0758 - val_accuracy: 0.9768
Epoch 2/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0651 - accuracy: 0.9805 - val_loss: 0.0667 - val_accuracy: 0.9804
Epoch 3/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0649 - accuracy: 0.9805 - val_loss: 0.0650 - val_accuracy: 0.9806
Epoch 4/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0620 - accuracy: 0.9817 - val_loss: 0.0679 - val_accuracy: 0.9793
Epoch 5/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0607 - accuracy: 0.9815 - val_loss: 0.0683 - val_accuracy: 0.9788


In [20]:
train_iteration(lr=2e-4, epochs=5)

=== Training with lr=0.0002 for 5 epochs ===
Epoch 1/5
1641/1641 [==============================] - 14s 7ms/step - loss: 0.0547 - accuracy: 0.9841 - val_loss: 0.0625 - val_accuracy: 0.9820
Epoch 2/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0529 - accuracy: 0.9845 - val_loss: 0.0607 - val_accuracy: 0.9820
Epoch 3/5
1641/1641 [==============================] - 9s 5ms/step - loss: 0.0522 - accuracy: 0.9849 - val_loss: 0.0621 - val_accuracy: 0.9818
Epoch 4/5
1641/1641 [==============================] - 10s 6ms/step - loss: 0.0519 - accuracy: 0.9849 - val_loss: 0.0601 - val_accuracy: 0.9827
Epoch 5/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0514 - accuracy: 0.9852 - val_loss: 0.0603 - val_accuracy: 0.9826


In [21]:
train_iteration(lr=2e-5, epochs=30)

=== Training with lr=2e-05 for 30 epochs ===
Epoch 1/30
1641/1641 [==============================] - 16s 8ms/step - loss: 0.0498 - accuracy: 0.9857 - val_loss: 0.0593 - val_accuracy: 0.9828
Epoch 2/30
1641/1641 [==============================] - 10s 6ms/step - loss: 0.0494 - accuracy: 0.9859 - val_loss: 0.0592 - val_accuracy: 0.9831
Epoch 3/30
1641/1641 [==============================] - 12s 7ms/step - loss: 0.0492 - accuracy: 0.9860 - val_loss: 0.0591 - val_accuracy: 0.9829
Epoch 4/30
1641/1641 [==============================] - 11s 7ms/step - loss: 0.0491 - accuracy: 0.9861 - val_loss: 0.0591 - val_accuracy: 0.9829
Epoch 5/30
1641/1641 [==============================] - 12s 7ms/step - loss: 0.0490 - accuracy: 0.9861 - val_loss: 0.0591 - val_accuracy: 0.9831
Epoch 6/30
1641/1641 [==============================] - 11s 6ms/step - loss: 0.0489 - accuracy: 0.9861 - val_loss: 0.0590 - val_accuracy: 0.9831
Epoch 7/30
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0489 - 

In [22]:
train_iteration(lr=1e-5, epochs=10)

=== Training with lr=1e-05 for 10 epochs ===
Epoch 1/10
1641/1641 [==============================] - 17s 9ms/step - loss: 0.0476 - accuracy: 0.9868 - val_loss: 0.0585 - val_accuracy: 0.9834
Epoch 2/10
1641/1641 [==============================] - 13s 8ms/step - loss: 0.0476 - accuracy: 0.9868 - val_loss: 0.0585 - val_accuracy: 0.9834
Epoch 3/10
1641/1641 [==============================] - 12s 7ms/step - loss: 0.0475 - accuracy: 0.9868 - val_loss: 0.0585 - val_accuracy: 0.9834
Epoch 4/10
1641/1641 [==============================] - 9s 5ms/step - loss: 0.0475 - accuracy: 0.9868 - val_loss: 0.0585 - val_accuracy: 0.9833
Epoch 5/10
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0475 - accuracy: 0.9868 - val_loss: 0.0585 - val_accuracy: 0.9834
Epoch 6/10
1641/1641 [==============================] - 11s 7ms/step - loss: 0.0475 - accuracy: 0.9867 - val_loss: 0.0585 - val_accuracy: 0.9833
Epoch 7/10
1641/1641 [==============================] - 9s 5ms/step - loss: 0.0475 - ac

In [23]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")

task name: axion1
INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v4.0/axion1_cnn/assets


INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v4.0/axion1_cnn/assets
